In [1]:
!pip install boto3

In [2]:
import pyspark
import boto3

from io import StringIO 
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType 
from pyspark.sql.types import ArrayType, DoubleType, BooleanType
from pyspark.sql.functions import col,array_contains

In [3]:
# Import SparkSession
from pyspark.sql import SparkSession

# Criando uma sessão com o Spark que existe localmente(atualmente configurado junto com o JupyterLab)
spark = SparkSession.builder \
      .master("local[1]") \
      .appName("AwariAula08") \
      .getOrCreate() 

In [14]:
df = spark.read.option("header", True).csv("./estados.csv")
print(df.show(), df.printSchema())

+---------+---+-------------------+--------+---------+--------+
|codigo_uf| uf|               nome|latitude|longitude|  regiao|
+---------+---+-------------------+--------+---------+--------+
|       11| RO|           Rondônia|  -10.83|   -63.34|   Norte|
|       12| AC|               Acre|   -8.77|   -70.55|   Norte|
|       13| AM|           Amazonas|   -3.47|    -65.1|   Norte|
|       14| RR|            Roraima|    1.99|   -61.33|   Norte|
|       15| PA|               Pará|   -3.79|   -52.48|   Norte|
|       16| AP|              Amapá|    1.41|   -51.77|   Norte|
|       17| TO|          Tocantins|   -9.46|   -48.26|   Norte|
|       21| MA|           Maranhão|   -5.42|   -45.44|Nordeste|
|       22| PI|              Piauí|    -6.6|   -42.28|Nordeste|
|       23| CE|              Ceará|    -5.2|   -39.53|Nordeste|
|       24| RN|Rio Grande do Norte|   -5.81|   -36.59|Nordeste|
|       25| PB|            Paraíba|   -7.28|   -36.72|Nordeste|
|       26| PE|         Pernambuco|   -8

In [16]:
df_json = spark.read.option("header", True).json("./estados.json")
print(df_json.show(), df_json.printSchema())

+---------+--------+---------+-------------------+--------+---+
|codigo_uf|latitude|longitude|               nome|  regiao| uf|
+---------+--------+---------+-------------------+--------+---+
|       11|  -10.83|   -63.34|           Rondônia|   Norte| RO|
|       12|   -8.77|   -70.55|               Acre|   Norte| AC|
|       13|   -3.47|    -65.1|           Amazonas|   Norte| AM|
|       14|    1.99|   -61.33|            Roraima|   Norte| RR|
|       15|   -3.79|   -52.48|               Pará|   Norte| PA|
|       16|    1.41|   -51.77|              Amapá|   Norte| AP|
|       17|   -9.46|   -48.26|          Tocantins|   Norte| TO|
|       21|   -5.42|   -45.44|           Maranhão|Nordeste| MA|
|       22|    -6.6|   -42.28|              Piauí|Nordeste| PI|
|       23|    -5.2|   -39.53|              Ceará|Nordeste| CE|
|       24|   -5.81|   -36.59|Rio Grande do Norte|Nordeste| RN|
|       25|   -7.28|   -36.72|            Paraíba|Nordeste| PB|
|       26|   -8.38|   -37.86|         P

In [11]:
spark.stop()

conf = pyspark.SparkConf()

# Criando uma sessão com o Spark que existe localmente(atualmente configurado junto com o JupyterLab)
conf.setMaster("local[1]") 
conf.set("spark.driver.host", "awari-jupyterlab") \
    .set("spark.sql.sources.commitProtocolClass", "org.apache.spark.sql.execution.datasources.SQLHadoopMapReduceCommitProtocol") \
    .set("parquet.enable.summary-metadata", "false") \
    .set("mapreduce.fileoutputcommitter.marksuccessfuljobs", "false") \
    .set("spark.driver.port", "20020") \
    .set("spark.hadoop.fs.s3a.endpoint", 'awari-nginx:9000') \
    .set("spark.hadoop.fs.s3a.endpoint.region", 'sa-east-1') \
    .set("spark.hadoop.fs.s3a.access.key", 'hxhFDOAzIu8nZM33') \
    .set("spark.hadoop.fs.s3a.secret.key", 'TKHENO4SuG4mLURfD0sxKXGvPokV9GT1') \
    .set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") \
    .set("spark.hadoop.fs.s3a.connection.ssl.enabled", "false") \
    .set("spark.hadoop.com.amazonaws.services.s3.enableV2", "true") \
    .set("spark.hadoop.fs.s3a.committer.staging.conflict-mode", "replace") \
    .set("spark.hadoop.fs.s3a.fast.upload", True) \
    .set("spark.hadoop.fs.s3a.path.style.access", True) \
    .set("spark.hadoop.fs.s3a.committer.name", "directory") \
    .set("spark.hadoop.fs.s3a.committer.staging.tmp.path", "/tmp/staging")

conf.setAppName('AwariAula08-S33')
sc = pyspark.SparkContext(conf=conf)

spark = SparkSession(sc)

In [12]:
client = boto3.client('s3', 
    endpoint_url='http://awari-minio-nginx:9000',
    aws_access_key_id='hxhFDOAzIu8nZM33',
    aws_secret_access_key='TKHENO4SuG4mLURfD0sxKXGvPokV9GT1',
    aws_session_token=None,
    config=boto3.session.Config(signature_version='s3v4'),
    verify=False,
    region_name='sa-east-1'
)

In [17]:
df.write.save("s3a://aula-08/estados", format="json") #de csv para json
df_json.write.save("s3a://aula-08/estados", format="csv") #de json para csv
df.write.save("s3a://aula-08/estados", format="parquet") #de csv para parquet

Py4JJavaError: An error occurred while calling o205.save.
: java.nio.file.AccessDeniedException: s3a://aula-08/estados: org.apache.hadoop.fs.s3a.auth.NoAuthWithAWSException: No AWS Credentials provided by TemporaryAWSCredentialsProvider SimpleAWSCredentialsProvider EnvironmentVariableCredentialsProvider IAMInstanceCredentialsProvider : com.amazonaws.SdkClientException: Unable to load AWS credentials from environment variables (AWS_ACCESS_KEY_ID (or AWS_ACCESS_KEY) and AWS_SECRET_KEY (or AWS_SECRET_ACCESS_KEY))
	at org.apache.hadoop.fs.s3a.S3AUtils.translateException(S3AUtils.java:212)
	at org.apache.hadoop.fs.s3a.S3AUtils.translateException(S3AUtils.java:175)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.s3GetFileStatus(S3AFileSystem.java:3799)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.innerGetFileStatus(S3AFileSystem.java:3688)
	at org.apache.hadoop.fs.s3a.S3AFileSystem.lambda$exists$34(S3AFileSystem.java:4703)
	at org.apache.hadoop.fs.statistics.impl.IOStatisticsBinding.lambda$trackDur

In [ ]:
df.write.mode("overwrite").format("json").save("s3a://aula-08/municipios")